In [1]:
# Importa as bibliotecas necessárias
%run '~/git/Bibliotecas.ipynb'

### 1.Fertilizantes  

In [2]:
# Carrega a base de dados contendo os registros de importação de fertilizantes
i=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/imp1.csv').sort_values(by=['CO_ANO','CO_MES'])
i.head()

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB,VL_FRETE,VL_SEGURO
0,1997,1,31010000,10,386,SP,1,817800,15222,15222,34324,1323,177
1,1997,1,31010000,10,573,SP,1,817800,1800,20000,7169,2638,0
2,1997,1,31021010,10,850,RR,7,260151,640,32000,7360,1120,0
3,1997,1,31021090,10,850,RR,7,260151,96000,96000,22080,3360,0
4,1997,1,31022100,10,23,GO,1,817800,1900000,1900000,184300,19000,0


Visualizando a tabela podemos intuir:  

* Colunas ordinais:
    * CO_ANO: ano do regitro
    * CO_MES: mês do registro

* Colunas categóricas:
    * CO_ANO e CO_MES: além do valor ordinal, essas colunas representam também valor categórico, ligados ao clima do momento indicaddo.
    * CO_NCM: NCM-Nomenclatura Comum do Mercosul, que identifica o produto com base no Sistema Internacional (SH) da Organização Mundial do Comércio, acrescido de mais dois digitos extras, que especificam ainda mais as características das mercadorias importadas.
    * CO_UNID: unidade monetária de valor utilizada no registro, que é identificada textualmente em tabela auxiliar.
    * CO_PAIS: país de origem, que é identificado textualmente em tabela auxiliar.
    * CO_VIA:  via física de transporte, que é identificada textualmente em tabela auxiliar.
    * CO_URF:  Unidade da Receita Federal, que é ligada ao um porto de desembarque, que é identificada textualmente em tabela auxiliar.
    * SG_UF_NCM: contém a sigla unidade federativa de destino da importação, que é identificada textualmente em tabela auxiliar.

  
* Colunas quantitativas:
    * QT_ESTAT: contém a quantidade de produto importado em unidade tipicamente utilizada no país de origem, que pode ser diferente do padrão principal, que é o quilograma.
    * KG_LIQUIDO contém o peso da importação em quilograma
    * VL_FOB quantifica o valor monetário da mercadoria na modalidade [FOB](https://pt.wikipedia.org/wiki/Free_on_Board#:~:text=Na%20modalidade%20FOB%2C%20o%20remetente,do%20seguro%20a%20partir%20da%C3%AD.)
    * VL_FRETE quantifica o valor do transporte da quantidade importada, do país de origem até o Brasil.
    * VL_SEGURO quantifica o valor do seguro sobre a mercadoria durante o transporte do país de origem até o Brasil.

In [3]:
i.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100430 entries, 0 to 100429
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   CO_ANO      100430 non-null  int64 
 1   CO_MES      100430 non-null  int64 
 2   CO_NCM      100430 non-null  int64 
 3   CO_UNID     100430 non-null  int64 
 4   CO_PAIS     100430 non-null  int64 
 5   SG_UF_NCM   100430 non-null  object
 6   CO_VIA      100430 non-null  int64 
 7   CO_URF      100430 non-null  int64 
 8   QT_ESTAT    100430 non-null  int64 
 9   KG_LIQUIDO  100430 non-null  int64 
 10  VL_FOB      100430 non-null  int64 
 11  VL_FRETE    100430 non-null  int64 
 12  VL_SEGURO   100430 non-null  int64 
dtypes: int64(12), object(1)
memory usage: 10.0+ MB


In [4]:
print(f'É {np.min(l)+len(l)-1==np.max(l)} que os anos estão contínuos')

NameError: name 'l' is not defined

In [ ]:
k=list(i.groupby('CO_ANO').CO_MES.unique().reset_index().CO_MES.values)

for K in range(len(k)):
    w=sorted(list(k.pop()),reverse=False)
    print(w)

Os meses estão corretos e contínuos.

In [ ]:
i.duplicated().sum()

Não há valores duplicados.

In [ ]:
i.duplicated(subset=['CO_ANO','CO_MES','CO_NCM','CO_PAIS','CO_URF','SG_UF_NCM','CO_VIA']).sum()

Não há valores duplicados no subconjunto de categorias, e se pode concluir com isso que cada linha da tabela representa uma importação única.

### Filtragem de caracteríticas exigidas no projeto:  

Considerando que todos os registros são importações de fertilizantes, as características de interesse são:  
* desembarques ocorridos através dos portos de Salvador e Aratú, no Estado da Bahia
* importações executadas por empresas sediadas no Estado da Bahia

Para identificar essas características listadas acima, serão filtrados registros onde:  

* as Unidades da Receita Federal brasileira, receptoras da importação,  contenham em seu nome alusão aos nomes dos portos, no caso, Aratu e Salvador.
* as Unidades Federativas da empresa importadora contenham a sigla do Estado da Bahia, BA.

In [ ]:
# Carrega tabela auxiliar com o nome das URF
urf=pd.read_csv('~/git/BD/Comexstat/URF.csv',encoding='Latin-1',sep=';')

# Identifica as URF que contenham no nome os nomes dos portos desejados, Aratu e Salvador.
urf_salvador_aratu=urf[(urf.NO_URF.str.contains('ARATU'))|(urf.NO_URF.str.contains('SALVADOR'))].CO_URF

# Filtra importações ligadas as URF identificadas e destinadas ao estado da Bahia
i=i[(i.CO_URF.isin(urf_salvador_aratu))&(i.SG_UF_NCM=='BA')]

### Seleciona as colunas de interesse desta fase do projeto:

As colunas de ano e mês são de interesse e serão mantidas.

As colunas utilizadas nas filtragens acima, CO_URF e SG_NCM_UF, tiveram a lista de seus possíveis valores reduzida, ficando estas colunas sem importância e por isso serão ambas descartadas.

In [ ]:
i.drop(columns=['CO_URF','SG_UF_NCM'],inplace=True)

Nessa primeira tentativa do projeto, em nome da simplicidade, não serão diferenciados os fertilizantes por NCM, sendo todos considerados um único produto simplesmente denominado "fertilizante". Assim, a coluna referente ao código NCM perde importância e não será mantida.

In [ ]:
i.drop(columns='CO_NCM',inplace=True)

O projeto não tem plano de utilizar os dados sobre o país de origem e essa coluna será descartada.

In [ ]:
i.drop(columns='CO_PAIS',inplace=True)

A coluna sobre a via de transporte apresenta praticamente todos os valores ligados a via de código "1". Assim, somente registros ligados a essa via serão mantidos e a coluna será descartada.

In [ ]:
i.groupby('CO_VIA')['KG_LIQUIDO'].sum().sort_values()

In [ ]:
i=i[i.CO_VIA==1]

Antes de descarta a coluna referente a via de transporte internacional, será feita uma conferência em relação a essa via ser igual ao conhecimento geral, que atribui a essa via ser marítima. Para isso será utilizada a tabela auxiliar que permite identificar nominalmente a via 1.

In [ ]:
via=pd.read_csv('~/git/BD/Comexstat/VIA.csv', encoding='Latin-1',sep=';')
via.head(1)

In [ ]:
print(f'É "{via[via.CO_VIA==1].NO_VIA.str.contains("MARITIMA").values[0]}" que a via de código "1" é marítima.')

In [ ]:
i.drop(columns='CO_VIA',inplace=True)

Abaixo podemos constatar que todos os registros de importação utilizam a unidade quantitativa de código 10. Assim a coluna fica sem utilidade e será descartada. Porém , antes do descarte, confirmaremos que a unidade de valor é expressa e quilograma, como acreditamos ser.

In [ ]:
i.groupby('CO_UNID')['KG_LIQUIDO'].sum().sort_values()

In [ ]:
unid=pd.read_csv('~/git/BD/Comexstat/NCM_UNIDADE.csv', encoding='Latin-1',sep=';')
unid.head(1)

In [ ]:
print(f'É "{unid[unid.CO_UNID==10].NO_UNID.str.contains("QUILOGRAMA").values[0]}" que a unidade de código "10" é quilograma.')

In [ ]:
i.drop(columns='CO_UNID',inplace=True)

A coluna 'QT_STAT' representa a quantidade importada expressa em unidades utilizadas no país de origem, que podem ser diferentes de quilograma. Isso não é de interesse do projeto e a coluna também será descartada.

In [ ]:
i.drop(columns='QT_ESTAT', inplace=True)

As colunas com valores monetários não são do interesse deste projeto e serã todas descartadas.

In [ ]:
i.drop(columns=['VL_FOB','VL_FRETE','VL_SEGURO'],inplace=True)

Após eliminações de tantas colunas categóricas  a tabela restante perde diferenciação entre registros e passa a apresentar duplicatas:

In [ ]:
i.duplicated(subset=['CO_ANO','CO_MES']).sum()

Procederemos o agrupamento nas categorias remanecentes, passando os registros a indicar a importação total de fertilizantes em um único mês de cada ano.

In [ ]:
i=i.groupby(by=['CO_ANO','CO_MES']).KG_LIQUIDO.sum().reset_index()
print(f"O número de duplicatas após o reagrupamento é de {i.duplicated(subset=['CO_ANO','CO_MES']).sum()}.")

### Validação dos pesos declarados

Os pesos em quilograma são todos maiores do que zero.

In [ ]:
i[i.KG_LIQUIDO<=0]

Vejamos a normalidade da distribuição dos pesos em quilograma registrados:

In [ ]:
g=sns.displot(height=2, aspect=1.618,data=i,x='KG_LIQUIDO',log_scale=True)
g.fig.suptitle('Importação de Fertilizantes na Bahia',y=1.1)
plt.show()

Os valores menores do que 10**6.3 são considerados anormais e serão descartados.

In [ ]:
i=i[i.KG_LIQUIDO>10**6.3]

Um registro apresenta peso muito acima da normalidade, próximo de 10**9 KG_LIQUIDO.

In [ ]:
i[i.KG_LIQUIDO==i.KG_LIQUIDO.max()]

Ele será substituido por interpolação dos valores do mesmo mês porém do ano anterior e do ano posterior.

In [ ]:
CO_ANO_erro=i[i.KG_LIQUIDO==i.KG_LIQUIDO.max()].CO_ANO.values[0]
CO_ANO_corr_anterior=i[i.CO_ANO==CO_ANO_erro+1].CO_ANO.values[0]
CO_ANO_corr_posterior=i[i.CO_ANO==CO_ANO_erro-1].CO_ANO.values[0]
mes_erro=i[i.KG_LIQUIDO==i.KG_LIQUIDO.max()].CO_MES.values[0]
val_corr=i[i.CO_ANO.isin([CO_ANO_corr_anterior,CO_ANO_corr_posterior])].KG_LIQUIDO.mean()
i.loc[i.KG_LIQUIDO==i.KG_LIQUIDO.max(),'KG_LIQUIDO']=val_corr

A distribição dos pesos registrados ficou como a seguir e foi considerada normal.

In [ ]:
sns.displot(height=2, aspect=1.618,data=i,x='KG_LIQUIDO',log_scale=True, bins=6).fig.suptitle('Importação de Fertilizantes na Bahia',y=1.1)
plt.show()

In [ ]:
# Cria luna com a data completa e no formato apropriado
i['date']=pd.to_datetime(i.CO_ANO.astype(str)+'/'+i.CO_MES.astype(str)+'/1')

Para conferência ampla, a seguir podemos ver as ocorrências por mês ao longo dos anos.

In [ ]:
g=sns.relplot(height=2, aspect=1.618,data=i,x='date',y='KG_LIQUIDO',col='CO_MES',col_wrap=6)
g.fig.suptitle('Importação de Fertilizantes na Bahia',y=1.1)
#g.set_xticklabels(rotation=90)
plt.tight_layout()

A tabela e transformada para o formato de martiz esparsa, o que permitirá ela ser fundida adiante com a tabela de chuvas, que passará pelo mesmo processo, evitando a perda de registros.

In [ ]:
# Martiz Esparsa
i_ini=i.date.min()
i_fim=i.date.max()
i[['CO_ANO','CO_MES']]=i[['CO_ANO','CO_MES']].astype('category')
i2=i.groupby(by=['CO_ANO','CO_MES']).KG_LIQUIDO.sum().reset_index()
i2['date']=pd.to_datetime(i2.CO_ANO.astype(str)+'/'+i2.CO_MES.astype(str)+'/1')
i2=i2[(i2.date>=i_ini)&(i2.date<=i_fim)]

In [ ]:
i=i2

In [ ]:
i.to_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/i.csv',index=False)

### 2.Chuvas

In [ ]:
# Importa as bibliotecas necessárias
%run '~/git/Bibliotecas.ipynb'

In [ ]:
# Carrega a tabela com histórico das chuvas no oeste baiano
c=pd.read_csv('/home/andre301267/git/Pricing-Fertilizer-take-Rains-into-account/temp/chuvas1.csv')

Seleciona colunas de interesse e unifica colunas que estão em duplicidade.

In [ ]:
c=c[c.columns[[20,0,21,1,22,2]]]
c.columns=['Estação','date1','date2','hora1','hora2','mm']
c.date1=pd.to_datetime(c.date1+'-'+c.hora1)
c.date2=pd.to_datetime(c.date2+'-'+c.hora2)
c['date']=pd.to_datetime(pd.concat([c.date1.dropna(),c.date2.dropna()]), utc=True)
c['CO_ANO']=c.date.dt.year
c['CO_MES']=c.date.dt.month
c['Dia']=c.date.dt.day
c['Hora']=c.date.dt.hour
c=c[c.columns[[0,6,7,8,9,10,5]]]
c.head(1)

Substitui valores inválidos, transforma formatação numérica para o formato desejado.

In [ ]:
c.loc[c.mm=='-9999','mm']=np.nan
c.mm=c.mm.str.replace(',','.')
c.mm=c.mm.astype('float')

Não há valores duplicados.

In [ ]:
c.duplicated().sum()

Há aproximadamente cem mil falhas nos registros de chuvas por hora.

In [ ]:
c.isna().sum()

Os registros com falha e sem falhas são separados em tabelas próprias.

In [ ]:
c_na=c[(c.mm.isna())]
c_not_na=c[~(c.mm.isna())]

In [ ]:
print(f"As importações com falha no registro de chuvas correspondem a {round(100*len(c_na)/len(c))} % do total")

A seguir se modifica a tabela com falhas no registro de forma a permitir a análise gráfica da distribuição das falhas ao longo das categorias existentes, com objetivo de verificar se há algum relacionamento lógico para as ocorrências.

In [ ]:
w=pd.concat([\
c_na.groupby('CO_ANO' ).CO_MES.count().reset_index().rename(columns={'CO_MES':'count','CO_ANO':' '}).assign(tipo='CO_ANO' ),
c_na.groupby('CO_MES' ).CO_ANO.count().reset_index().rename(columns={'CO_ANO':'count','CO_MES':' '}).assign(tipo='CO_MES' ),
c_na.groupby('Dia'    ).CO_ANO.count().reset_index().rename(columns={'CO_ANO':'count','Dia':' '}).assign(tipo='Dia'    ),
c_na.groupby('Hora'   ).CO_ANO.count().reset_index().rename(columns={'CO_ANO':'count','Hora':' '}).assign(tipo='Hora'   ),
c_na.groupby('Estação').CO_ANO.count().reset_index().rename(columns={'CO_ANO':'count','Estação':' '}).assign(tipo='Estação')])

for t in w.tipo.unique():
    w_t=w[w.tipo==t]
    sns.catplot(height=1,aspect=4*1.6,data=w_t,x=' ',y='count',kind='bar').fig.suptitle(t,y=1.1)
    plt.xticks(rotation=90)
    if t =='Estação':
        plt.xticks(rotation=22.5)
    elif t=='CO_MES':
        plt.xticks(rotation=0)
print(f"\n\nApenas {round(100*len(c_not_na[c_not_na.mm!=0])/len(c_not_na))} % apresentam ocorrência de chuva. Os demais registros apresentam ausência de chuva.\n\
Esse fato faz com que a mediana seja sempre igual a zero, impedindo o uso dela para substituir as centenas de milhares de valores nulos\n\
encontrados na tabela.")

print('\n\n\n\n                  Horas com Falha no Registro de Chuvas\n\n\n')
plt.show()

print('\nConclsão: As falhas no registro de chuva aparentam estar normalmente distribuidas entre as categorias,\
\nnão sendo identificado relação entre elas.\n\n\n\n')

A seguir analizaremos a normalidade da distribuição dos registros de chuva. A cada aprovação de normalidade, se elimina a menor subdivisão de tempo, reagrupando, até chegar na subdvisão de tempo por mês, que é a medida de tempo foco deste projeto.

In [ ]:
# Cria nova coluna com volume de chuva em unidade logarítmica
c_not_na['mm_hora_log']=np.log10(c_not_na.mm)

In [ ]:
# Plota volumes logarítmos de chuva por hora em gráfico com escala logarítmica (duplo logaritmo)
sns.displot(height=2, aspect=1.618,data=c_not_na[c_not_na.mm_hora_log!=0],x='mm_hora_log',log_scale=True, bins=4).fig.suptitle('Chuvas no Oeste Baiano',y=1.1)
plt.show()

A distribuição de dos volumes de chuva por hora foram considerados normais.
A seguir se agrupam-se os valores por dia.

In [ ]:
c_dia=c_not_na.groupby(by=c_not_na.columns[[0,1,2,3,4]].tolist())[['mm']].sum().reset_index()
c_dia.head(1)

Recria-se a coluna com volume de chuva logarítmico.

In [ ]:
c_dia['mm_dia_log']=np.log10(c_dia.mm)

In [ ]:
# Plota-se novamente os volumes logarítmicos de chuva em gráfico com escala logarítmica
sns.displot(height=2, aspect=1.618,data=c_dia[c_dia.mm_dia_log!=0],x='mm_dia_log',log_scale=True, bins=4).fig.suptitle('Chuvas no Oeste Baiano',y=1.1)
plt.show()

Nos dois gráficos apresentados acima podemos notar:  

> Devido a grande amplitude nos volumes de chuvas registrados, para possibilitar a visualização da curva normal, foi necessário aplicar duas vezes seguidas a transformação logarítmica aos volumes de chuva:
> * primeiro: transformando diretamente o valor medido para logarítmo
> * segundo: utilizando no gráfico a escala logarítmica

Agrupa-se a tabela por mês

In [ ]:
c_mes=c_dia.groupby(by=['Estação','CO_ANO','CO_MES']).mm.sum().reset_index().rename(columns={'mm':'mm_mes'})

Com os agrupamentos até aqui procedidos, não é mais necessário a dupla transofrmação logaritmica, mas ainda é necessário a escala logarítmica no gráfico, para se visualizar a curva normal d distribuição de chuvas por mês.

In [ ]:
sns.displot(height=2, aspect=1.618,data=c_mes[c_mes.mm_mes!=0],x='mm_mes',log_scale=True, bins=4).fig.suptitle('Chuvas no Oeste Baiano',y=1.1)
plt.show()

Para simplificar, levaremos em conta apenas um linha do tempo de registros de chuva, sendo a seguir eleiminada a subdivisão referente as estações meteorológicas e agrupando os dados utilizando os valores medianos entre as estações.

In [ ]:
print(f"\nApós realizar todos os agrupamentos, {round(100*len(c_mes[c_mes.mm_mes!=0])/len(c_mes))} % apresentam ocorrência de chuva e os demais registros apresentam ausência de chuva.\n\
Esse fato faz com que a mediana seja em sua maioria maior que zero, permitindo o uso dela para a conjunção dos dados entre estações.\n\n")

In [ ]:
c=c_mes.groupby(by=['CO_ANO','CO_MES']).mm_mes.median().reset_index()

In [ ]:
sns.displot(height=2, aspect=1.618,data=c[c.mm_mes!=0],x='mm_mes',log_scale=True,bins=4).fig.suptitle('Falhas no Registro de Chuvas',y=1.1)
plt.show()

Após realizar todos os agrupamentos necessários até alcançar a unidade de tempo desejada, por fim, os dados são considerados normais.

In [ ]:
# Cria coluna com data completa e no formato adequado
c['date']=pd.to_datetime(c.CO_ANO.astype(str)+'/'+c.CO_MES.astype(str)+'/1')

In [ ]:
# Cria matriz esparsa, com o mesmo objetivo do que foi feito com a tabela de imortações
c_ini=c.date.iloc[0]
c_fim=c.date.iloc[-1]
c[['CO_ANO','CO_MES']]=c[['CO_ANO','CO_MES']].astype('category')
c=c.groupby(by=['CO_ANO','CO_MES']).mm_mes.sum().reset_index()
c[['CO_ANO','CO_MES']]=c[['CO_ANO','CO_MES']].astype('int')
c['date']=pd.to_datetime(c.CO_ANO.astype(str)+'/'+c.CO_MES.astype(str)+'/1')
c=c[(c.date>=c_ini)&(c.date<=c_fim)]
c=c.rename(columns={'mm_mes':'mm'})
c=c.sort_values(by=['date'])
c

Análise de tendência delongo prazo nos volums de chuva

In [ ]:
sns.lmplot(data=c,x='CO_ANO',y='mm')

Considerando o intervalo de confiança se pode dizer que não há alta confiança para se afirmar que há tendência de longo prazo diferente de zero. Assim, essa etapa não será executada para as chuvas.

In [ ]:
c.to_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/c.csv',index=False)

### 3. Relação entre Importações e Chuvas - Visualização e Cálculo

In [ ]:
# Importa as bibliotecas necessárias
%run '~/git/Bibliotecas.ipynb'
from sklearn.preprocessing import MinMaxScaler as mms
mms=mms()
from sklearn import metrics

In [ ]:
# Carrega as tabelas
i=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/i.csv')
c=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/c.csv')

# Funde as tabelas
i_c=i.merge(c)

# Seleciona e ordena as colunas
i_c=i_c[i_c.columns[[3,0,1,2,4]]]
i_c.date=pd.to_datetime(i_c.date)
i_c.head(1)

1ª Tentaiva: Simplesmente plotandoos valores reais

In [ ]:
icl=pd.concat([
    i_c[['date','KG_LIQUIDO']].rename(columns={'KG_LIQUIDO':'valor'}).assign(tipo='fert_import'),
    i_c[['date','mm'        ]].rename(columns={'mm'        :'valor'}).assign(tipo='chuva')
])                         
sns.relplot(data=icl,x='date',y='valor',hue='tipo', kind='line',height=2,aspect=3*1.6).fig.suptitle('Bahia')
plt.show()

Quando simplesmente comparamos as chuvas com as importações, como nessa primeira tentativa, percebemos que os dados e chuva ficam achatados na parte inferior do gráfico, sem possibilitar visualizar as variações. Isso ocorre em função da diferença de escala, que será resolvida a seguir, com o reescalonamento dos valores para uma faixa de 0 a 1, com zero sendo o menor e 1 o maior valor de cada fonte de dado.  

Outra diferença entre os valores é que as importações apresentam uma tendência visual de crescimento ao longo dos anos, fato que não ocorre com as chuvas, já que o clima é mais constante do que a economia no longo prazo. Isso também será resolvido calculando-se a tendência central dos fertilizantes, que é a regressão linear, e extraindo-se ela dos valores.

2ª Tentaiva: Extraindo tendência dos fertilizantes e rescalonando todosos valores

In [ ]:
i_c['kg_tend']=np.poly1d(np.polyfit(i_c.date.astype(int),i_c.KG_LIQUIDO, deg=2))(i_c.date.astype(int))
i_c['kg_flat']=i_c.KG_LIQUIDO/i_c.kg_tend*i_c.kg_tend.min()

In [ ]:
i_c[['fert_import_scaled','chuva_scaled']]=mms.fit_transform(i_c[['kg_flat','mm']])

In [ ]:
icl=pd.concat([
    i_c[['date','fert_import_scaled']].rename(columns={'fert_import_scaled':'valor'}).assign(tipo='fert_import_scaled'),
    i_c[['date','chuva_scaled'      ]].rename(columns={'chuva_scaled':'valor'      }).assign(tipo='chuva_scaled'      )
              ]) 
score=round(metrics.mean_absolute_error(i_c.fert_import_scaled,i_c.chuva_scaled),2)

sns.relplot(data=icl,x='date',y='valor',hue='tipo', kind='line',height=2,aspect=3*1.6).fig.suptitle(f'Bahia - chuva sem adiantamento, mae_score={score}.')
plt.show()

Nesta tentativa acima podemos ver as variações de chuvas e fertilizantes numa mesma escala e já podemos notar semelhança no formato delas, o que indica que estamos "no caminho certo". A distância vertical média entre os registros de chuva e importação ou, a média do erro absoluto (mae_score) foi determinada, e será base de comparação numérica com as tentaivas a seguir.

Esse projeto se propõe, de certa forma, a responder se as curvas das chuvas acima induzem ou não as curvas de importação, numa relação de causa e efeito. Um fato que causa o outro deve acontecer antes do outro, além de ter magnitude proporcional. Então, para comparar a chuva com as importações, a seguir, repetidamente, os registros de chuva terão sua data alterada, somando-se a ela 1 mês a cada avaliação, buscando sobrepor as curvas o tanto quanto possível. Quando essa sobreposição for máxima, a média de erro entre uma curva e outra alcançará o seu menor valor.

3ª tentaiva: Adiantando as chuvas nos meses até antes completar 1 ano de avanço

In [ ]:
q=pd.DataFrame()

def ad_plot(df):

    global q
    
    global i_c
    
    i_c=ad(df)
    
    score=round(metrics.mean_absolute_error(i_c.fert_import_scaled,i_c.chuva_scaled),2)

    r=pd.DataFrame({'Meses de Antecedência das Chuvas':[adi],'mae_score entre fertilizante e chuva':[score]})

    q=pd.concat([q,r])
    
    icl=pd.concat([df[['date','fert_import_scaled']].rename(columns={'fert_import_scaled':'valor'}).assign(tipo='fert_import_scaled'),
                   df[['date','chuva_scaled']].rename(columns={'chuva_scaled':'valor'}).assign(tipo='chuva_scaled')]) 
    
    sns.relplot(data=icl,x='date',y='valor',hue='tipo', kind='line',height=2,aspect=3*1.6)\
    .fig.suptitle(f'Bahia - Registro de Chuva Adiantado em {adi} mês, mae_score={score}')
    
    plt.show()

In [ ]:
def ad(df):
    ic_f=df.drop(columns=['mm','chuva_scaled'])
    ic_c=df.drop(columns=['KG_LIQUIDO','kg_tend','kg_flat','fert_import_scaled'])
    ic_c.CO_MES=df.CO_MES+1
    ic_c.loc[ic_c.CO_MES==13,'CO_ANO']=ic_c[ic_c.CO_MES==13].CO_ANO+1
    ic_c.loc[ic_c.CO_MES==13,'CO_MES']=1
    ic_c.date=pd.to_datetime(ic_c.CO_ANO.astype(str)+'/'+ic_c.CO_MES.astype(str)+'/1')
    df=ic_c.merge(ic_f)
    return df

In [ ]:
for adi in range(1,11):
    ad_plot(i_c)
sns.relplot(data=q,x='Meses de Antecedência das Chuvas',y='mae_score entre fertilizante e chuva',kind='line', height=2,aspect=1.6)\
.fig.suptitle('Chuvas e Desembarque de Fertilizantes na Bahia',y=1.1)
plt.show()

Pode-se comparar visualmente nos gráficos acima que a melhor sobreposição das curvas foi com 7 meses de avanço dos registros de chuva. Ainda , pode-se ver que o erro foi sendo reduzido gradativamente desde o 1º mẽs de avanço das chuvas.  

Em outras palavras, parece que as chuvas já são capazes de induzir movimentos de importação mesmo após poucos meses de ocorridas, e vão aumentando o poder de indução cada vez mais, até o 7º mês após sua ocorrência, quando a partir dai começa a declinar esse poder.

### Conclusão  

O volume de chuva induz parcialmente a chegada de importações de fertilizantes, atingindo máxima indução aos 7 meses após a precipitação.